In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo('i1').min and c_max < np.iinfo('i1').max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo('i2').min and c_max < np.iinfo('i2').max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo('i4').min and c_max < np.iinfo('i4').max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print("Data usage is less about", round(start_mem - end_mem, 2), "Mb (-", round(100* (start_mem - end_mem) / start_mem, 1), "%)")
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[(energy["building_id"]<100)]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)
del buildings
del weather
energy = reduce_memory_usage(energy)
print (energy.info())

Data usage is less about 56.89 Mb (- 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864557 entries, 0 to 864556
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           864557 non-null  datetime64[ns]
 1   site_id             864557 non-null  int8          
 2   building_id         864557 non-null  int8          
 3   meter_reading       864557 non-null  float16       
 4   primary_use         864557 non-null  category      
 5   air_temperature     864263 non-null  float16       
 6   cloud_coverage      487693 non-null  float16       
 7   dew_temperature     864263 non-null  float16       
 8   precip_depth_1_hr   864459 non-null  float16       
 9   sea_level_pressure  856210 non-null  float16       
 10  wind_direction      839970 non-null  float16       
 11  wind_speed          864557 non-null  float16       
dtypes: category(1), datetime64[ns](1), float1

In [4]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

In [5]:
energy_train, energy_test = train_test_split(energy[(energy["meter_reading"]>0)], test_size=0.2)
print (energy_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 427872 entries, 569868 to 642339
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           427872 non-null  datetime64[ns]
 1   site_id             427872 non-null  int8          
 2   building_id         427872 non-null  int8          
 3   meter_reading       427872 non-null  float16       
 4   primary_use         427872 non-null  category      
 5   air_temperature     427868 non-null  float16       
 6   cloud_coverage      248992 non-null  float16       
 7   dew_temperature     427868 non-null  float16       
 8   precip_depth_1_hr   427869 non-null  float16       
 9   sea_level_pressure  425610 non-null  float16       
 10  wind_direction      414109 non-null  float16       
 11  wind_speed          427872 non-null  float16       
 12  hour                427872 non-null  int8          
 13  weekday             427872 no

In [6]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_lr = [[]]*len(buildings)
for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_lr[building].append([0]*(len(lr_columns)-3))
        energy_train_bh = pd.DataFrame(energy_train_b[energy_train_b["hour"]==hour])
        y = energy_train_bh["meter_reading_log"]
        if len(y) > 0:
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            energy_lr[building][hour] = model.coef_
            energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)
print (energy_lr[0])

[array([-0.11992739,  5.46253347,  5.43377131,  5.48291016,  5.42603687,
        5.48245608,  5.46731771,  5.4359976 ,  0.        ]), array([-0.09109574,  5.45036674,  5.44230769,  5.46814904,  5.51536258,
        5.46000565,  5.43625   ,  5.46928267,  0.        ]), array([-0.15260988,  5.45950592,  5.45675223,  5.51732337,  5.50952244,
        5.46485293,  5.45242746,  5.4334375 ,  0.        ]), array([-0.11545281,  5.45323779,  5.47496449,  5.50540865,  5.43970637,
        5.46555561,  5.50094289,  5.45963542,  0.        ]), array([-0.15806563,  5.461853  ,  5.44167565,  5.49344308,  5.47678257,
        5.44841794,  5.44070871,  5.44516226,  0.        ]), array([-0.17168142,  5.5246452 ,  5.48737981,  5.47952586,  5.46685755,
        5.46106771,  5.44086746,  5.38266782,  0.        ]), array([-0.11814056,  5.44879687,  5.46484375,  5.47251157,  5.46758473,
        5.44607118,  5.45616319,  5.42352764,  0.        ]), array([-0.09622519,  5.49167137,  5.49140625,  5.4914363 ,  5.445240

In [7]:
sites = range(0, energy["site_id"].max() + 1)
primary_uses = energy["primary_use"].unique()
lr_columns_use = ["meter_reading_log", "hour", "building_id",
                  "is_holiday", "primary_use", "site_id"]
for wday in range(0,7):
    lr_columns_use.append("is_wday" + str(wday))
energy_lr_use = {}
energy_lr_use_site = {}
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns_use)
for primary_use in primary_uses:
    energy_train_u = energy_train_lr[energy_train_lr["primary_use"]==primary_use]
    if len(energy_train_u) > 0:
        energy_lr_use_site[primary_use] = [[]]*len(sites)
        for site in sites:
            energy_lr_use_site[primary_use][site] = [[]]*len(hours)
            energy_train_us = energy_train_u[energy_train_u["site_id"]==site]
            if len(energy_train_us) > 0:
                for hour in hours:
                    energy_train_uth = energy_train_us[energy_train_us["hour"]==hour]
                    y = energy_train_uth["meter_reading_log"]
                    if len(y) > 0:
                        x = energy_train_uth.drop(labels=["meter_reading_log",
                            "hour", "building_id", "site_id",
                            "primary_use"], axis=1)
                        model = LinearRegression(fit_intercept=False).fit(x, y)
                        energy_lr_use_site[primary_use][site][hour] = model.coef_
                        energy_lr_use_site[primary_use][site][hour] = np.append(energy_lr_use_site[primary_use][site][hour], model.intercept_)
        energy_lr_use[primary_use] = [[]]*len(hours)
        for hour in hours:
            energy_train_th = energy_train_u[energy_train_u["hour"]==hour]
            y = energy_train_th["meter_reading_log"]
            if len(y) > 0:
                x = energy_train_th.drop(labels=["meter_reading_log",
                    "hour", "building_id", "site_id", "primary_use"], axis=1)
                model = LinearRegression(fit_intercept=False).fit(x, y)
                energy_lr_use[primary_use][hour] = model.coef_
                energy_lr_use[primary_use][hour] = np.append(energy_lr_use[primary_use][hour], model.intercept_)
print (energy_lr_use_site)

{'Education': [[array([-0.14921935,  5.67416306,  5.67173794,  5.65342218,  5.70757712,
        5.67077318,  5.65519033,  5.61997109,  0.        ]), array([-0.06460325,  5.64483736,  5.6390366 ,  5.6307695 ,  5.61646687,
        5.66355244,  5.6162845 ,  5.62945741,  0.        ]), array([0.00843857, 5.59086825, 5.64984425, 5.62806525, 5.63967728,
       5.63644072, 5.60628539, 5.53701465, 0.        ]), array([-0.13388347,  5.62130209,  5.66170396,  5.61516557,  5.62474457,
        5.61900913,  5.66843518,  5.56289776,  0.        ]), array([-0.16690194,  5.59626853,  5.65501868,  5.65321738,  5.60322988,
        5.63146592,  5.5920046 ,  5.56856462,  0.        ]), array([-0.02081277,  5.61094334,  5.6260978 ,  5.67635879,  5.65430328,
        5.64921548,  5.61812883,  5.61285959,  0.        ]), array([-0.06511228,  5.73694161,  5.76177078,  5.68556675,  5.74890674,
        5.75028199,  5.68596073,  5.70813571,  0.        ]), array([-0.03744111,  5.80850973,  5.87141578,  5.86214126,  5.

In [8]:
def calculate_model (x):
    lr = -1
    model = energy_lr[x.building_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use_site[x.primary_use][x.site_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use[x.primary_use][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Linear regression quality, 100 buildings:",
       energy_test_lr_rmsle)

Linear regression quality, 100 buildings: 0.34476821099834737
